# Modern Data Structures Homework 3

Hsin-Ju (Lulu) Huang  
hh3105

In [28]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from math import sqrt
from random import randint

## Question 1
### [10 pts] Import the data as a pd.DataFrame object. Use one or more functions from the pandas library to split the data into three unique datasets. The first should include columns that are factors only (i.e., categorical data), the second should include columns that are numeric only, and the third should include columns with logical/boolean variables only.

In [29]:
# Import the data as pd.DataFrame object
housing_prices = pd.read_csv(r"C:\Users\lulu\Documents\GitHub\QMSS-GR5072-Spring2025_Class\HW\HW04\Housing_prices_data.csv")

# Split dataset by data type
housing_categorical = housing_prices.select_dtypes(include=['object'])  # Categorical (factor) data
housing_numeric = housing_prices.select_dtypes(include=['int64', 'float64'])  # Numeric data
housing_boolean = housing_prices.select_dtypes(include=['bool'])  # Boolean data

## Question 2
### [10 pts] Using the second dataset from question #1, for each numeric variable use the apply function from the pandas library to return the mean, median and standard deviation (SD) of all numeric variables.

In [30]:
# Define a function to compute mean, median, and standard deviation
def compute_statistics(series):
    return pd.Series({
        'Mean': series.mean(),
        'Median': series.median(),
        'Std Dev': series.std()
    })

# Apply the function to each numeric column
summary_stats = numeric_df.apply(compute_statistics)

# Display the results
summary_stats

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
Median,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
Std Dev,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883


## Question 3
### [10 pts] Using the second dataset from question #1, create a new dataset that only includes the variable indicating sales price ("SalePrice"). Search for a categorical variable which has between 2-4 categories in the full dataset that you think may be related to the sales price of houses. Give the mean, median and SD for SalePrice based on the different groups of the categorical variables. Does the average price seem to vary by the different levels of the categorical factor you chose?

In [31]:
categorical_var = 'PoolQC'  
housing_prices_filtered = housing_prices[[categorical_var, 'SalePrice']].dropna()
grouped_stats = housing_prices_filtered.groupby(categorical_var)['SalePrice'].agg(['mean', 'median', 'std'])
print(grouped_stats)

            mean    median            std
PoolQC                                   
Ex      490000.0  490000.0  360624.458405
Fa      215500.0  215500.0   48790.367902
Gd      201990.0  171000.0   63441.392639


Yes, the average price seems to vary by the different levels of the pool quality, with Ex being the highest and Gd being the lowest.

## Question 4
### [5 pts] Regress "SalePrice" on "LotArea", "OverallQual", "OverallCond" with the statsmodels library. Use the syntax below to replicate the output shown you are able to estimate this regression model.

In [32]:
# Define independent variables
X = housing_numeric[['LotArea', 'OverallQual', 'OverallCond']]
X = sm.add_constant(X)  # Add intercept

# Define dependent variable
y = housing_numeric['SalePrice']

# Fit OLS regression model
model = sm.OLS(y, X).fit()

# Display the results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     935.9
Date:                Wed, 26 Mar 2025   Prob (F-statistic):               0.00
Time:                        00:42:19   Log-Likelihood:                -17760.
No. Observations:                1460   AIC:                         3.553e+04
Df Residuals:                    1456   BIC:                         3.555e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.022e+05   8633.073    -11.832      

## Question 5
### [10 pts] Write a for loop that loops through each level of the categorical factor you chose in Q3, where each iteration of the for loop subsets your data for one group of the categorical variable at a time, and then fits the same regression model specified above in Q4 k times, where k refers to the number of unique categories. On each iteration of your for loop, store the model.summary() information as a new element in a list object. Once your for loop is done running, print the list. There should be k regressions which you estimated. Does the effect of LotArea on SalePrice vary within the different levels of the categorical factor you chose? What is another, more common way to answer a question like this (i.e., a question about moderation or effect heterogeneity)? Hint: Don't forget to account for the standard error when comparing the regression slopes!

In [34]:
# Define the categorical variable (PoolQC in this case)
categorical_var = 'PoolQC'

# Drop NA values for relevant columns
housing_prices_filtered = housing_prices[[categorical_var, 'SalePrice', 'LotArea', 'OverallQual', 'OverallCond']].dropna()

# Get unique categories of PoolQC (excluding NaN)
categories = housing_prices_filtered[categorical_var].unique()

# Create an empty list to store the regression summaries
regression_summaries = []

# Loop through each category of the categorical variable 'PoolQC'
for category in categories:
    # Subset the data for the current category
    subset_data = housing_prices_filtered[housing_prices_filtered[categorical_var] == category]
    
    # Define independent variables (add intercept)
    X = subset_data[['LotArea', 'OverallQual', 'OverallCond']]
    X = sm.add_constant(X)
    
    # Define the dependent variable
    y = subset_data['SalePrice']
    
    # Fit the OLS regression model
    model = sm.OLS(y, X).fit()
    
    # Store the regression summary in the list
    regression_summaries.append((category, model.summary()))

# Print the regression summaries for each category
for category, summary in regression_summaries:
    print(f"Regression results for {category}:")
    print(summary)
    print("\n\n")

Regression results for Ex:
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 26 Mar 2025   Prob (F-statistic):                nan
Time:                        00:47:28   Log-Likelihood:                 32.306
No. Observations:                   2   AIC:                            -60.61
Df Residuals:                       0   BIC:                            -63.23
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        7133.4392 

C:\Users\lulu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\lulu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\lulu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\lulu\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\regression\linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\lulu\AppDa

## Question 6
### [15 pts] Using the first dataset from question #1, for each categorical variable use a for loop to print a frequency and relative frequency table for every variable. To make your output readable, for each variable you should print the following in this order, placing the variable name where you see blanks below.

In [39]:
# Looping through each category (column) in the housing_categorical DataFrame
for category in housing_categorical.columns:
    # Creating a frequency table for each category, excluding NaN values
    frequency_table = housing_categorical[category].value_counts(dropna=True)
    
    # Creating a relative frequency table (proportions) for each category
    relative_frequency_table = housing_categorical[category].value_counts(normalize=True, dropna=True)
    
    # Setting the name attribute of the frequency tables to the column name
    frequency_table.name = category
    relative_frequency_table.name = category

    # Printing the frequency table for the current category
    print(f"Frequency Table for {category} ==================")
    print(frequency_table)

    # Printing the relative frequency table for the current category
    print(f"\nRelative Frequency Table for {category}")
    print(relative_frequency_table)
    
    # Adding some spacing 
    print("\n\n")


Frequency Table for MSZoning ==================
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

Relative Frequency Table for MSZoning
MSZoning
RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64



Frequency Table for Street ==================
Street
Pave    1454
Grvl       6
Name: Street, dtype: int64

Relative Frequency Table for Street
Street
Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64



Frequency Table for Alley ==================
Alley
Grvl    50
Pave    41
Name: Alley, dtype: int64

Relative Frequency Table for Alley
Alley
Grvl    0.549451
Pave    0.450549
Name: Alley, dtype: float64



Frequency Table for LotShape ==================
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Relative Frequency Table for LotShape
LotShape
Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3   

## Question 7
### [10 pts] Do the same thing as number 6, but with a while loop this time.

In [40]:
# Initialize the index for iterating through columns
index = 0

# Get the number of columns in the DataFrame
num_columns = len(housing_categorical.columns)

# Using a while loop to iterate through each category (column) in the housing_categorical DataFrame
while index < num_columns:
    # Get the column name at the current index
    category = housing_categorical.columns[index]
    
    # Creating a frequency table for each category, excluding NaN values
    frequency_table = housing_categorical[category].value_counts(dropna=True)
    
    # Creating a relative frequency table (proportions) for each category
    relative_frequency_table = housing_categorical[category].value_counts(normalize=True, dropna=True)
    
    # Setting the name attribute of the frequency tables to the column name
    frequency_table.name = category
    relative_frequency_table.name = category

    # Printing the frequency table for the current category
    print(f"Frequency Table for {category} ==================")
    print(frequency_table)

    # Printing the relative frequency table for the current category
    print(f"\nRelative Frequency Table for {category}")
    print(relative_frequency_table)
    
    # Adding some spacing
    print("\n\n")
    
    # Increment the index to move to the next column
    index += 1


Frequency Table for MSZoning ==================
MSZoning
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

Relative Frequency Table for MSZoning
MSZoning
RL         0.788356
RM         0.149315
FV         0.044521
RH         0.010959
C (all)    0.006849
Name: MSZoning, dtype: float64



Frequency Table for Street ==================
Street
Pave    1454
Grvl       6
Name: Street, dtype: int64

Relative Frequency Table for Street
Street
Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64



Frequency Table for Alley ==================
Alley
Grvl    50
Pave    41
Name: Alley, dtype: int64

Relative Frequency Table for Alley
Alley
Grvl    0.549451
Pave    0.450549
Name: Alley, dtype: float64



Frequency Table for LotShape ==================
LotShape
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Relative Frequency Table for LotShape
LotShape
Reg    0.633562
IR1    0.331507
IR2    0.028082
IR3   

## Question 8
### [15 pts] Using the Rocket class which we created as part of the week 7 class activity, we are going to create a new class called shuttle. The shuttle class should inherit all of Rocket's characteristics but has one additional parameter: flights_completed which measures the number of flights the shuttle has completed.

In [39]:
# Import the GED dataset (assuming the file is in CSV format)
ged_data = pd.read_csv(r"C:\Users\lulu\Documents\GitHub\QMSS-GR5072-Spring2025_Class\HW\HW03\ged_data.csv")  # Replace with actual path

# Call the t_test function
results = t_test(ged_data['income_log'], ged_data['ged'])

# Print the results
print(results)

The homogeneity of variance assumption was not violated, so an independent samples t-test was conducted.
  Continuous Variable Binary Variable  Total Sample Size  Mean Difference  \
0          income_log             ged               5976            -0.48   

   SE of Mean Difference    DF  t-statistic P-value  \
0                   0.06  5974       -7.458    .000   

                         Test  
0  Independent samples t-test  


## Question 9
### [15 pts] Write a single for loop which randomly creates 11 rockets and 11 shuttles using randomly generated integers. Store this output in two separate lists. Once you have your data, use for loops to print the messages you see below. Note you can use the randint() function from the random module (which you can import with the command from random import randint) to randomly generate the characteristics of your Rockets and Shuttles.